In [91]:
using ForwardDiff
using ReverseDiff
using Base.Test

In [13]:
f = (x, w) -> sum(x .* w)

g = (x, w) -> ForwardDiff.gradient(x -> f(x, w), x)


@show f(x, w)

@show g(x, w)

# ReverseDiff.jacobian(w -> g(x, w), w)

f(x,w) = 11.0
g(x,w) = [3.0,4.0]


2-element Array{Float64,1}:
 3.0
 4.0

In [14]:
ForwardDiff.jacobian(w -> g(x, w), w)

2×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0

In [15]:
g(x, w)

2-element Array{Float64,1}:
 3.0
 4.0

In [119]:
function viewblocks{T <: NTuple}(data::AbstractArray, shapes::AbstractVector{T})
    starts = cumsum(vcat([1], prod.(shapes)))
    [reshape(view(data, starts[i]:(starts[i+1] - 1)), shapes[i]) for i in 1:length(shapes)]
end

viewblocks (generic function with 3 methods)

In [120]:
@inferred viewblocks(w, shapes)

2-element Array{Base.ReshapedArray{Float64,2,SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true},Tuple{}},1}:
 [0.2 0.3]
 [0.5]    

In [121]:
f = (x, flat_weights, shapes) -> begin
    weights = viewblocks(flat_weights, shapes)
    y = x
    J = eye(length(x))
    for i in 1:(length(shapes) - 1)
        w = weights[i]
        y = w * y
        J = w * J
        z = y .>= 0
        y = y .* z
        J = J .* z
    end
    w = weights[end]
    w * y, w * J
end


x = [1.0, 2.0]
shapes = [(1, 2), (1, 1)]
w = [0.2, 0.3, 0.5]

3-element Array{Float64,1}:
 0.2
 0.3
 0.5

In [122]:
f(x, w, shapes)

([0.4],
[0.1 0.15])

In [95]:
ForwardDiff.jacobian(x -> f(x, w, shapes)[1], x)

1×2 Array{Float64,2}:
 0.1  0.15

In [96]:
ReverseDiff.jacobian(x -> f(x, w, shapes)[1], x)

1×2 Array{Float64,2}:
 0.1  0.15

In [97]:
ReverseDiff.jacobian(w -> f(x, w, shapes)[2], w)

2×3 Array{Float64,2}:
 0.5  0.0  0.2
 0.0  0.5  0.3

In [98]:
(f(x, w + [0, 0, 0.001], shapes)[2] - f(x, w, shapes)[2]) / 0.001

1×2 Array{Float64,2}:
 0.2  0.3